In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.mnist
# ! Mnist is already integrated to TF :-) 
# In fact it's integrated in Keras dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# x = image , y = label

In [ ]:
print(len(x_train))
print(len(x_test))
# Just to ensure we have the right dataset
# 60000 train images , 10000 test images

In [ ]:
x_train[0] # Array of images (28x28 array)
# (0 = black, 255 = white opposite to EMNIST !!!)

In [ ]:
plt.imshow(x_train[0], cmap='gray')


In [ ]:
print(y_train[0])

In [ ]:
plt.imshow(x_test[0], cmap='gray')

In [ ]:
# let's play with layers (thanks to Keras !)
# Since TF 2.0 Keras is intimatly mixed to TF
# And devs encourages to use more Keras functions than just pure TF
# convolution , flatten, dense
from tensorflow.keras.layers import Conv2D, Flatten, Dense
# we need to discuss of this too 
from tensorflow.keras import Model

In [ ]:
# cf Conv2D Keras docs fo full list of arguments
# Conv2D(filters, kernel_size, activation)
# Flatten() turn a 2d images into a 1d array of 784 pixels (28*28)
# Dense(neuros, activation = 'relu or softmax at the very least')

class MNISTModel(Model):
    def __init__(self):
        super (MNISTModel, self).__init__()
        self.conv1 = Conv2D(32,3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        #(32 --> 128 for every outputs we have 4 neurons)
        # 128 we may increase the precision but time increases too
        self.dense2 = Dense(10, activation='softmax')
        # we have 10 digits so 10 neurons
        # and format encoding is [0 0 0 0 0 0 0 0  1 0 ]
        # read the course pdf !!!
        
    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self.dense1(x2)
        return self.dense2(x3)
    
model = MNISTModel()


In [ ]:
# loss function & optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
# Caterogical losses
optimizer = tf.keras.optimizers.Adam() #Instead of simple gradient descent optimizer


In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [ ]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(inputs, outputs): # correct answers
    with tf.GradientTape() as tape: # Change the Bias 
        predictions = model.call(inputs)
        loss = loss_function(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # tests
    train-loss(loss) # should decrese quickly
    train_accuracy(output, predictions)
        

In [ ]:
@tf.function
def test_step(inputs, outputs): # correct answers
    predictions = model.call(inputs)
    loss = loss_function(outputs, predictions)
    
    
    train-loss(loss) # should decrese quickly
    train_accuracy(output, predictions)

In [ ]:
# formatting and normalizing the dataset
x_train, x_test = x_train / 255.0, x_test / 255.0
# add a new dimension to the array
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
# now we have 4 dimensions (read tf docs )

In [ ]:
x_train[0]

In [ ]:
# let's shuffle !!!
# this is specially helpfull depending on how we collected the data
# Ex. all the 0 then all the 1 and so on
# Shuffling add a bit of randomness, anyway our model we'll  learn only 0 then 1
train_data = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_data = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

In [1]:
# Warning dûe to a bug in Jupyter notebook we need to copu everything here
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_data = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

class MNISTModel(Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(10, activation='softmax')

    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self.dense1(x2)
        return self.dense2(x3)
    
model = MNISTModel()

loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_function(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(outputs, predictions)
    
@tf.function
def test_step(inputs, outputs):
    predictions = model(inputs)
    loss = loss_function(outputs, predictions)
    
    test_loss(loss)
    test_accuracy(outputs, predictions)
    
epochs = 5

for epoch in range(epochs):
    for train_inputs, train_labels in train_data:
        train_step(train_inputs, train_labels)
        
    for test_inputs, test_labels in test_data:
        test_step(test_inputs, test_labels)
        
    template = 'Epochs: {}, Train loss: {}, Train accuracy: {}, Test loss: {} Test accuracy: {}'
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result(),
        test_loss.result(),
        test_accuracy.result(),
    ))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

2021-11-19 22:02:02.743147: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2021-11-19 22:02:02.766251: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2491860000 Hz
2021-11-19 22:02:02.767020: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5650b692e2d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-19 22:02:02.767093: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-19 22:02:02.767521: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 1